## Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import pyodbc 

## Configurations

In [ ]:
server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = '{username}'
password = '{password}'   
driver= '{ODBC Driver 17 for SQL Server}'

## Functions

In [ ]:
#https://covid19.who.int/WHO-COVID-19-global-data.csv

# Data extraction functions

def downloadFile(filename, download_url):
    req = requests.get(download_url)
    with open(filename, 'wb') as file:
        file.write(req.content) 

In [ ]:
# Data transformation functions

# clean column names
def fixColumnNames(input_df):
    columns = []
    for column in input_df.columns:
        column = column.title()
        column = column.replace(" " , "")
        column = column.replace("_" , "")
        column = column.replace("," , "")
        column = column.replace(";" , "")
        column = column.replace("{" , "")
        column = column.replace("}" , "")  
        column = column.replace("(" , "")
        column = column.replace(")" , "")
        column = column.replace("\t", "")
        column = column.replace("=" , "")
        columns.append(column)
    return columns

# check the max length of each column
def checkColumnLen(input_df):
    length_dict = {}
    for column in input_df.columns:
        length_dict[column] = [max(input_df[column].astype(str).apply(len)), min(input_df[column].astype(str).apply(len))]
    return length_dict

def changeDataType(input_df, column, data_type):
    input_df[column] = input_df[column].astype(data_type)

## Extract

In [ ]:
#downloadFile('WHO-COVID-19-global-data.csv','https://covid19.who.int/WHO-COVID-19-global-data.csv')

## Transform

In [ ]:
# create DataFrame

df = pd.read_csv('WHO-COVID-19-global-data.csv')

In [ ]:
# clean column names

df.columns = fixColumnNames(df)

In [ ]:
# change data types

changeDataType(df, 'DateReported', 'datetime64')

In [ ]:
# check if any 'CountryCode' is missing
df['Country'].loc[df['CountryCode'].isna()].value_counts()

# add Namibia's short code
df.loc[df['Country'] == 'Namibia', 'CountryCode'] = 'NA'

In [ ]:
# create 'Key' column by concatenating 'CountryCode' and 'DateReported'

df['Key'] = df['CountryCode'] + df['DateReported'].astype(str).str.replace('-','')

In [ ]:
# set 'Key' as an index and ensure that there are no duplicates

# df.set_index('Key', inplace=True, verify_integrity=True)

In [ ]:
# test

df_test = df.head(1000)

## Load

In [ ]:
conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()



for index, row in df_test.iterrows():
     cursor.execute("INSERT INTO dbo.WHO_Covid19 (DateReported, CountryCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) values(?,?,?,?,?,?,?)", row.DateReported, row.CountryCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
conn.commit()
cursor.close()